In [0]:
%pip install databricks-vectorsearch-preview
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from datasets import load_dataset , Dataset, concatenate_datasets 
import pandas as pd
import sqlite3

In [0]:
from databricks.vector_search.client import VectorSearchClient
client = VectorSearchClient()

In [0]:
%sql
-- CREATE A CATALOG, SCHEMA AND VOLUME TO STORE DATA NEEDED FOR THIS. IN PRACTICE, YOU COULD USE AN EXISTING VOLUME
CREATE CATALOG IF NOT EXISTS structured_rag;
USE CATALOG structured_rag;
CREATE DATABASE IF NOT EXISTS structured_rag_db;
USE DATABASE structured_rag_db;
CREATE VOLUME IF NOT EXISTS structured_rag_raw;

download the soccer player sqlite database from the following kaggle page and upload to the volume we just created (drag drop or programmatically persist): https://www.kaggle.com/datasets/hugomathien/soccer/data

In [0]:
conn = sqlite3.connect('/Volumes/structured_rag/structured_rag_db/structured_rag_raw/database.sqlite')

In [0]:
# Query to fetch the table names from the SQLite master table.
query = "SELECT name FROM sqlite_master WHERE type='table';"
# Execute the query and fetch all table names.
table_names = conn.execute(query).fetchall()
table_names.pop(0)
table_names

[('Player_Attributes',),
 ('Player',),
 ('Match',),
 ('League',),
 ('Country',),
 ('Team',),
 ('Team_Attributes',)]

In [0]:
for table in table_names:
  table_name = table[0]
  
  # Query to fetch all data from the current table.
  query = f"SELECT * FROM {table_name};"
  
  # Use Pandas to read data from the SQLite database into a DataFrame.
  df = pd.read_sql_query(query, conn)
  spark.createDataFrame(df).write.saveAsTable(table_name)

In [0]:
# Use spark SQL to show all tables in the specified database.
tables = spark.sql(f"SHOW TABLES")

# Display the list of tables.
tables.show()

+-----------------+-----------------+-----------+
|         database|        tableName|isTemporary|
+-----------------+-----------------+-----------+
|structured_rag_db|          country|      false|
|structured_rag_db|           league|      false|
|structured_rag_db|            match|      false|
|structured_rag_db|           player|      false|
|structured_rag_db|player_attributes|      false|
|structured_rag_db|             team|      false|
|structured_rag_db|  team_attributes|      false|
+-----------------+-----------------+-----------+



In [0]:
# Get a list of table names in the schema
tables = spark.sql("SHOW TABLES")
table_names = [row.tableName for row in tables.collect()]
table_names

['country',
 'league',
 'match',
 'player',
 'player_attributes',
 'team',
 'team_attributes']

In [0]:
# Initialize an empty list to store table descriptions
table_descriptions = []

# Loop over each table
for table_name in table_names:
    # Describe the table
    table_description = spark.sql(f"DESCRIBE {table_name}")
    
    # Create a sentence describing the table
    sentence = f"Table '{table_name}' has the following columns:"
    
    # Loop over columns in the table description
    for row in table_description.collect():
        # Extract column name and data type
        column_name, data_type, _ = row
        sentence += f"\n- Column '{column_name}' with data type '{data_type}'"
    
    # Add the table description sentence to the list
    table_descriptions.append(sentence)

In [0]:
#Verify if it worked or not
table_descriptions[1]

"Table 'league' has the following columns:\n- Column 'id' with data type 'bigint'\n- Column 'country_id' with data type 'bigint'\n- Column 'name' with data type 'string'"

In [0]:
ids = list(range(7))
len(ids)== len(table_descriptions)

True

In [0]:
index_dict = {'ids': ids, 'table_descriptions': table_descriptions}
spark.createDataFrame(pd.DataFrame.from_dict(index_dict)).write.saveAsTable('table_descriptions')

In [0]:
%sql 
SELECT * FROM table_descriptions

ids table_descriptions 0 Table 'country' has the following columns:
- Column 'id' with data type 'bigint'
- Column 'name' with data type 'string' 1 Table 'league' has the following columns:
- Column 'id' with data type 'bigint'
- Column 'country_id' with data type 'bigint'
- Column 'name' with data type 'string' 2 Table 'match' has the following columns:
- Column 'id' with data type 'bigint'
- Column 'country_id' with data type 'bigint'
- Column 'league_id' with data type 'bigint'
- Column 'season' with data type 'string'
- Column 'stage' with data type 'bigint'
- Column 'date' with data type 'string'
- Column 'match_api_id' with data type 'bigint'
- Column 'home_team_api_id' with data type 'bigint'
- Column 'away_team_api_id' with data type 'bigint'
- Column 'home_team_goal' with data type 'bigint'
- Column 'away_team_goal' with data type 'bigint'
- Column 'home_player_X1' with data type 'double'
- Column 'home_player_X2' with data type 'double'
- Column 'home_player_X3' with data type 'double'
- Column 'home_player_X4' with data type 'double'
- Column 'home_player_X5' with data type 'double'
- Column 'home_player_X6' with data type 'double'
- Column 'home_player_X7' with data type 'double'
- Column 'home_player_X8' with data type 'double'
- Column 'home_player_X9' with data type 'double'
- Column 'home_player_X10' with data type 'double'
- Column 'home_player_X11' with data type 'double'
- Column 'away_player_X1' with data type 'double'
- Column 'away_player_X2' with data type 'double'
- Column 'away_player_X3' with data type 'double'
- Column 'away_player_X4' with data type 'double'
- Column 'away_player_X5' with data type 'double'
- Column 'away_player_X6' with data type 'double'
- Column 'away_player_X7' with data type 'double'
- Column 'away_player_X8' with data type 'double'
- Column 'away_player_X9' with data type 'double'
- Column 'away_player_X10' with data type 'double'
- Column 'away_player_X11' with data type 'double'
- Column 'home_player_Y1' with data type 'double'
- Column 'home_player_Y2' with data type 'double'
- Column 'home_player_Y3' with data type 'double'
- Column 'home_player_Y4' with data type 'double'
- Column 'home_player_Y5' with data type 'double'
- Column 'home_player_Y6' with data type 'double'
- Column 'home_player_Y7' with data type 'double'
- Column 'home_player_Y8' with data type 'double'
- Column 'home_player_Y9' with data type 'double'
- Column 'home_player_Y10' with data type 'double'
- Column 'home_player_Y11' with data type 'double'
- Column 'away_player_Y1' with data type 'double'
- Column 'away_player_Y2' with data type 'double'
- Column 'away_player_Y3' with data type 'double'
- Column 'away_player_Y4' with data type 'double'
- Column 'away_player_Y5' with data type 'double'
- Column 'away_player_Y6' with data type 'double'
- Column 'away_player_Y7' with data type 'double'
- Column 'away_player_Y8' with data type 'double'
- Column 'away_player_Y9' with data type 'double'
- Column 'away_player_Y10' with data type 'double'
- Column 'away_player_Y11' with data type 'double'
- Column 'home_player_1' with data type 'double'
- Column 'home_player_2' with data type 'double'
- Column 'home_player_3' with data type 'double'
- Column 'home_player_4' with data type 'double'
- Column 'home_player_5' with data type 'double'
- Column 'home_player_6' with data type 'double'
- Column 'home_player_7' with data type 'double'
- Column 'home_player_8' with data type 'double'
- Column 'home_player_9' with data type 'double'
- Column 'home_player_10' with data type 'double'
- Column 'home_player_11' with data type 'double'
- Column 'away_player_1' with data type 'double'
- Column 'away_player_2' with data type 'double'
- Column 'away_player_3' with data type 'double'
- Column 'away_player_4' with data type 'double'
- Column 'away_player_5' with data type 'double'
- Column 'away_player_6' with data type 'double'
- Column 'away_player_7' with data type 'double'
- Column 'away_player_8' with data type 'double

In [0]:
%sql
ALTER TABLE table_descriptions SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
client.create_index(
  source_table_name="structured_rag.structured_rag_db.table_descriptions",
  dest_index_name="vs_catalog.vs_schema.structuredrag-table-index",
  primary_key="ids",
  index_column="table_descriptions",
  embedding_model_endpoint_name="all-MiniLM-L6-v2-avi")

{'name': 'vs_catalog.vs_schema.structuredrag-table-index',
 'spec': {'continuous': {},
  'src_table': 'structured_rag.structured_rag_db.table_descriptions',
  'dest_index': 'vs_catalog.vs_schema.structuredrag-table-index',
  'primary_key': 'ids',
  'vector_index': {'column': 'table_descriptions',
   'embedding_model_endpoint_name': 'all-MiniLM-L6-v2-avi'}},
 'index_status': {'state': 'NOT_READY',
  'message': 'Index creation is in progress. Check latest status in Delta Live Tables: href="#joblist/pipelines/4e984a8a-8b67-4a29-ad6c-a1abe129fcb1/updates/9fcc44ad-31b0-40a1-8cf3-21e739771f24"'}}

In [0]:
index = client.list_indexes("vs_catalog")
index

{'indexes': [{'name': 'vs_catalog.chatbot.databricks_documentation_index',
   'index_definition': {'name': 'vs_catalog.chatbot.databricks_documentation_index',
    'spec': {'continuous': {},
     'src_table': 'dbdemos.chatbot.databricks_documentation',
     'dest_index': 'vs_catalog.chatbot.databricks_documentation_index',
     'primary_key': 'id',
     'vector_index': {'column': 'content'}}},
   'index_status': {'state': 'READY',
    'message': 'Index creation succeeded using Delta Live Tables: href="#joblist/pipelines/0c2fb52e-3289-4dd3-8c71-9c24a61d08cd/updates/800e624a-e111-42b6-8400-7dac19c3ada1"',
    'indexed_row_count': 2326}},
  {'name': 'vs_catalog.chatbot_ivy.databricks_documentation_index',
   'index_definition': {'name': 'vs_catalog.chatbot_ivy.databricks_documentation_index',
    'spec': {'continuous': {},
     'src_table': 'dbdemos.chatbot_ivy.databricks_documentation',
     'dest_index': 'vs_catalog.chatbot_ivy.databricks_documentation_index',
     'primary_key': 'id',


In [0]:
client.get_index('vs_catalog.vs_schema.structuredrag-table-index')

{'name': 'vs_catalog.vs_schema.structuredrag-table-index',
 'spec': {'continuous': {},
  'src_table': 'structured_rag.structured_rag_db.table_descriptions',
  'dest_index': 'vs_catalog.vs_schema.structuredrag-table-index',
  'primary_key': 'ids',
  'vector_index': {'column': 'table_descriptions',
   'embedding_model_endpoint_name': 'all-MiniLM-L6-v2-avi'}},
 'index_status': {'state': 'READY',
  'message': 'Index creation succeeded using Delta Live Tables: href="#joblist/pipelines/4e984a8a-8b67-4a29-ad6c-a1abe129fcb1/updates/9fcc44ad-31b0-40a1-8cf3-21e739771f24"',
  'indexed_row_count': 7}}

In [0]:
results = client.similarity_search(
  index_name = "vs_catalog.vs_schema.structuredrag-table-index",
  query_text = "Write a SQL query to  find the number of soccer players",
  columns = ["ids", "table_descriptions"], # columns to return
  num_results = 2)

results['result']['data_array'][0]

['2',
 "Table 'match' has the following columns:\n- Column 'id' with data type 'bigint'\n- Column 'country_id' with data type 'bigint'\n- Column 'league_id' with data type 'bigint'\n- Column 'season' with data type 'string'\n- Column 'stage' with data type 'bigint'\n- Column 'date' with data type 'string'\n- Column 'match_api_id' with data type 'bigint'\n- Column 'home_team_api_id' with data type 'bigint'\n- Column 'away_team_api_id' with data type 'bigint'\n- Column 'home_team_goal' with data type 'bigint'\n- Column 'away_team_goal' with data type 'bigint'\n- Column 'home_player_X1' with data type 'double'\n- Column 'home_player_X2' with data type 'double'\n- Column 'home_player_X3' with data type 'double'\n- Column 'home_player_X4' with data type 'double'\n- Column 'home_player_X5' with data type 'double'\n- Column 'home_player_X6' with data type 'double'\n- Column 'home_player_X7' with data type 'double'\n- Column 'home_player_X8' with data type 'double'\n- Column 'home_player_X9' w

In [0]:
context = results['result']['data_array'][0][1]+ ' \n '+results['result']['data_array'][1][1]
print(context)


Table 'match' has the following columns:
- Column 'id' with data type 'bigint'
- Column 'country_id' with data type 'bigint'
- Column 'league_id' with data type 'bigint'
- Column 'season' with data type 'string'
- Column 'stage' with data type 'bigint'
- Column 'date' with data type 'string'
- Column 'match_api_id' with data type 'bigint'
- Column 'home_team_api_id' with data type 'bigint'
- Column 'away_team_api_id' with data type 'bigint'
- Column 'home_team_goal' with data type 'bigint'
- Column 'away_team_goal' with data type 'bigint'
- Column 'home_player_X1' with data type 'double'
- Column 'home_player_X2' with data type 'double'
- Column 'home_player_X3' with data type 'double'
- Column 'home_player_X4' with data type 'double'
- Column 'home_player_X5' with data type 'double'
- Column 'home_player_X6' with data type 'double'
- Column 'home_player_X7' with data type 'double'
- Column 'home_player_X8' with data type 'double'
- Column 'home_player_X9' with data type 'double'
- Col

In [0]:
def fill_prompt(context: str, question: str) -> str:
    template = f"""[INST] <<SYS>>
You are an AI data analyst, helping business users by generating SQL queries based on their questions asked in English. 

Some information about possibly relevant tables will be provided to you, but you may or may not need all the tables provided for the SQL query.

Ensure the SQL queries you generate are accurate to the best of your ability

Please only print out the SQL query, nothing else

<</SYS>>

{context}

{question} [/INST]
"""
    return template

In [0]:
question  = 'What is the number of soccer players?'

In [0]:
prompt = fill_prompt(context, question)
prompt

"[INST] <<SYS>>\nYou are an AI data analyst, helping business users by generating SQL queries based on their questions asked in English. \n\nSome information about possibly relevant tables will be provided to you, but you may or may not need all the tables provided for the SQL query.\n\nEnsure the SQL queries you generate are accurate to the best of your ability\n\nPlease only print out the SQL query, nothing else\n\n<</SYS>>\n\nTable 'match' has the following columns:\n- Column 'id' with data type 'bigint'\n- Column 'country_id' with data type 'bigint'\n- Column 'league_id' with data type 'bigint'\n- Column 'season' with data type 'string'\n- Column 'stage' with data type 'bigint'\n- Column 'date' with data type 'string'\n- Column 'match_api_id' with data type 'bigint'\n- Column 'home_team_api_id' with data type 'bigint'\n- Column 'away_team_api_id' with data type 'bigint'\n- Column 'home_team_goal' with data type 'bigint'\n- Column 'away_team_goal' with data type 'bigint'\n- Column '

In [0]:
import mlflow.gateway
mlflow.gateway.set_gateway_uri("databricks")

In [0]:
# Query the completions route using the mlflow client

query= mlflow.gateway.query(
        route="mosaicml-llama2-70b-completions",
        data={
            "prompt": prompt,
            'temperature':0.0
        },
    )
query

{'candidates': [{'text': "\nSELECT COUNT(DISTINCT player_api_id)\nFROM player;\n\nThis query will return the number of unique soccer players in the 'player' table, based on the 'player_api_id' column.",
   'metadata': {}}],
 'metadata': {'model': 'llama2-70b-chat', 'route_type': 'llm/v1/completions'}}

In [0]:
query['candidates'][0]['text'].split(';')[0].replace('\n',' ')

' SELECT COUNT(DISTINCT player_api_id) FROM player'

In [0]:
display(spark.sql(query['candidates'][0]['text'].split(';')[0].replace('\n',' ')))

count(DISTINCT player_api_id)
11060
